In [13]:
import requests
from bs4 import BeautifulSoup
# to use webdriver you need chromedriver.exe in the same folder as the .py script
from selenium import webdriver
import time
import os
import csv
import random
from pikepdf import Pdf as PDF
from io import BytesIO
# returns list of links of all pdf on a page
def get_all_hrefs_of_pdfs(url,browser_instance, sleep_time=3):
    browser = browser_instance
    
    # get web page
    # print("before")
    browser.get(url)
    # execute script to scroll down the page( copy from internet )
    try:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        # time.sleep(1)
        browser.execute_script("window.scrollTo(0, 0);")
    except :
        pass
    # sleep for time
    # print("after")
    # time.sleep(sleep_time)
    page_source = browser.page_source
    soup = BeautifulSoup(page_source, 'html5lib')
    a_tags = soup.find_all(name="a")
    all_pdf_links_of_page = []
    if url[-4:]==".pdf":
        
        item_dict = {
            'title':url.split("/")[-1],
            'link':url,
        }
        all_pdf_links_of_page.append(item_dict)
    for a_tag in a_tags:
        try:
            if not a_tag.has_attr('href'):
                continue
            if a_tag["href"]=="" or a_tag["href"][0] not in "h/":
                continue
            if a_tag["href"][-3:] == "pdf":
                if a_tag["href"][0] == "/":
                    url_split = url.split("/")
                    mylink = "/".join(url_split[0:3]) + a_tag["href"]
                a_tag.text.replace("\n", " ")
                item_dict = {
                    'title':a_tag["href"].split("/")[-1],
                    'link':mylink,
                }
                all_pdf_links_of_page.append(item_dict)
        except :
            pass
    del a_tags
    
    return all_pdf_links_of_page

# Create and merge pdfs
# def create_and_merge_pdfs(links_list, start_page, end_page):
#     pdf_name_list_which_were_created = []
#     initial_start_page = start_page
#     for a_page in links_list:
#         pdf_ = PDF.new()
#         version = pdf_.pdf_version
#         if len(links_list[start_page-initial_start_page]) == 0:
#             print(f"---------Index {start_page} has no files")
#             pdf_.close()
#             start_page += 1
#             continue
#         for index_link, a_link in enumerate(a_page):
#             content = requests.get(a_link).content
#             with open(f"pdf_downloader_folder/{start_page}_{index_link}.pdf", 'wb') as my_data:
#                 my_data.write(content)
#                 my_data.close()
#             pdf_name_list_which_were_created.append(f"pdf_downloader_folder/{start_page}_{index_link}.pdf")
            
#             try:
                
#                 src = PDF.open(f"pdf_downloader_folder/{start_page}_{index_link}.pdf")
#                 version = max(version, src.pdf_version)
#                 pdf_.pages.extend(src.pages)
#                 src.close()
#                 print(f"Fetched & Merged PageNumber_PDFindex-- {start_page}_{index_link}")
#             except:
#                 print(f"------------{start_page}_{index_link}.pdf was not merged due to some reason!!!")
#                 print("----It is written to a separate file")
#                 with open(f"pdf_downloader_folder/Compiled_{start_page}_{index_link}_was.pdf", 'wb') as f:
#                     f.write(content)
#                     f.close()
#             time.sleep(random.random()*3)
#         pdf_.remove_unreferenced_resources()
#         pdf_.save(f"pdf_downloader_folder/Compiled_{start_page}_.pdf", min_version=version)
#         pdf_.close()
#         # merger.write(f"pdf_downloader_folder/Compiled_{start_page}_.pdf")
#         # merger.close()
#         start_page += 1

    
#     for pdf in pdf_name_list_which_were_created:
#         os.remove(pdf)
#     print("Compiled PDF Created")
#     return


def get_soup_from_BeautifulSoup(url="https://www.google.com/search", params = None , *headers):
    response = requests.get(url, params=params, headers=headers )
    # soup = BeautifulSoup(response.text, 'lxml')
    # with open("test.html", mode="w+", encoding='utf-8') as f:
    #     f.write(soup.prettify())
    #     f.close()
    return BeautifulSoup(response.text, 'html5lib')
# using Beautiful Soup
# The main content of google search
def get_data_list_using_BeautifulSoup(soup):
    main_div = soup.find("div", id="main")
    # making list of all div tags
    one_before_start_div = main_div.findChild("div")
    one_before_start_div = one_before_start_div.find_next_siblings("div")
    one_before_start_div = one_before_start_div[1]

    list_of_divs = one_before_start_div.find_next_siblings("div")
    # Removes last elements which has other searches (which div is not imp)
    list_of_divs.pop()
    # Clearing the memory
    # del one_before_start_div, main_div
    # Making list of all the data on the google search page
    all_info = []
    
    for the_div in list_of_divs:
        link_to_post = the_div.find_next("a").attrs["href"]
        if link_to_post[0] == "/":
            link_to_post = "https://www.google.com" + link_to_post

        info_list = the_div.find_next("div").find_next("div").find_next_sibling().find_next_sibling()
        info_list = info_list.find_next("span").parent.find_all(string=True)
        
        try:
            if info_list[0]=="People also ask" or info_list[0]=="Missing:  "or info_list[0]== "Related searches":
                continue
        except:
            pass
        title = the_div.find_next("h3").text
        # Removes an extra dot in list
        info_list.pop(1)
        info_list.insert(0, title)
        # words_date = info_list[0].split(" ")
        # if int(words_date[0]) > 4 and words_date[1] == "months": #If the news is from before 4 months it wont be saved into list
        #     continue
        # datetime_object = datetime.strptime(info_list[0], '%b %d %Y %I:%M%p')
        # print(datetime_object)
        # print(link_to_post)
        _info = [link_to_post]
        # Getting correct(Redirected URL)
        _info[0] = requests.get(_info[0],allow_redirects=False)
        _info[0]= _info[0].headers['location']
        
        _info.extend(info_list)
        
        dict_info = {
            "link_post":_info[0],
            "title":_info[1],
            "description":"",
            "publish_date":""
        }
        try :
            dict_info["description"]=_info[3]
            dict_info["publish_date"]=_info[2]
            dict_info["description"] = dict_info["description"].replace(u'\xa0', u' ')
        except :
            dict_info["description"]=_info[2]
            dict_info["description"] = dict_info["description"].replace(u'\xa0', u' ')
        # appending the info into the list
        all_info.append(dict_info)
        # order in final list is url, published at, description
    # print(len(all_info))
    return all_info
# all_info = get_data_list_using_BeautifulSoup(soup)
# all_info
# print(Extractor(extractor='ArticleExtractor', url=myurl.url).getText())

In [14]:

params = {
    "q": 'sustainability report of Bain and company :pdf',
    "hl": "en",
    # "tbm": "nws",
    "num": 100, #no. of results per page
    #  "tbs": "sbd:1", #Sort by Date
    "start": None,
}
page_number = 1 #change the page number from here
params["start"] = params["num"]*(page_number-1)
time.sleep(1.2)

google_search_results = get_data_list_using_BeautifulSoup(get_soup_from_BeautifulSoup(params=params))



In [15]:
a_set_titles = set()
a_set_link = set()
browser = webdriver.Chrome()
all_links_and_titles=[]
# pss= get_all_hrefs_of_pdfs("https://www.baincapital.com/citizenship/responsible-business", browser)
# cddd = get_all_hrefs_of_pdfs("https://www.baincapital.com/", browser)
# print(pss)
# print(cddd)
for index,card in enumerate(google_search_results):
    # try:
        liii = get_all_hrefs_of_pdfs(card["link_post"], browser)
        browser.get("https://www.google.com")
        for href_dict in liii:
            if href_dict["link"] not in a_set_link:
                all_links_and_titles.append(href_dict)
                a_set_link.add(href_dict['link'])
                print(f"Title: '{href_dict['title']}'")
                print(f'^^ {href_dict["link"]}')
        # print(index, "done")
    # except:
    #     pass
browser.close()
# print(len(a_set_link))
# import 
# open("pdf_list.csv")
# print(a_set_link)

'bain_gri_index_2020.pdf'
^^ https://www.bain.com/contentassets/6b37083d53dc4e9aa676993fa0c4c7dc/bain_gri_index_2020.pdf
'wef-metrics_brief.pdf'
^^ https://www.bain.com/contentassets/6b37083d53dc4e9aa676993fa0c4c7dc/wef-metrics_brief.pdf
'sustainable-procurement-factsheet.pdf'
^^ https://www.bain.com/contentassets/110ea77537cd4ebfb1aecc12d231c69d/sustainable-procurement-factsheet.pdf
'bain_brief_transforming_business_for_a_sustainable_economy.pdf'
^^ https://www.bain.com/contentassets/d72f85af23564454970916d56eb4b5b4/bain_brief_transforming_business_for_a_sustainable_economy.pdf
'bain-environment-policy_21.pdf'
^^ https://www.bain.com/contentassets/e47c807504b24e68ba28604ac22fd9bc/bain-environment-policy_21.pdf
'bain_brief_sustainability_is-the-next-digital.pdf'
^^ https://www.bain.com/globalassets/noindex/2020/bain_brief_sustainability_is-the-next-digital.pdf
'bain_brief_achieving_breakthrough_results_in_sustainability.pdf'
^^ https://www.bain.com/contentassets/bd8f5f0eaede4b5db624fb6

In [ ]:
# browser = webdriver.Chrome()
# get_all_hrefs_of_pdfs("https://www.baincapital.com/citizenship/responsible-business", browser_instance=browser)